In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import numpy as np
import pandas as pd

In [4]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import AbstractBaseUnit, MagnitudeTable, Hashes, HashRate, Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, ElectricalSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects, ProjectStatement
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer, ProjectStatementSerializer

In [5]:
import numpy as np
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage, \
    expected_difficulty, hashes_to_hash_rate, win_percentage


In [6]:
from drillbit.statements.statements import init_environment, ProjectTemplate, ROITemplate, \
    analysis

In [7]:
btc = BitcoinEnvironmentUtility()
env = Environment.objects.latest('created_at')
project = Project.objects.latest('created_at')

In [8]:
projects = Projects.objects.latest('created_at')

In [13]:
from tqdm.auto import tqdm, trange

In [20]:
def project_comparison_summary(env, projects):
    projects = Project.objects.filter(pk__in=projects)
    comps = []
    for project in tqdm(projects):
        data = {
            'environment': env.id, 
            'project': project.id, 
            'frequency': 'M'
        }
        ser = ProjectStatementSerializer(data=data, frequency='M')
        ser.is_valid()
        ser.save()
        comps.append(ser.data['profitability'])

    df = pd.DataFrame(comps).T
    df.columns = [p.name for p in projects]

    return df

In [21]:
df = project_comparison_summary(env, projects.projects.all())

100%|████████████████████████████████████████████████████| 2/2 [00:32<00:00, 16.12s/it]


In [22]:
df

,Project TEST #5,Project TEST #1
Capacity,40000000.0,40000000.0
Compute Power,37905709.547501,37905709.547501
Infra Power,2094290.452499,2094290.452499
Number of Rigs,11448.41726,9038.619266
Hash Rate,1602778416384802816.0,903861926617003392.0
Hash Rate per Rig,151108190137765.8125,127154245861491.421875
Total Hashes,132860130208063308764807168.0,88266149100979022947418112.0
Energy Consumption,2792727272727273.0,2820196721311475.0
Efficiency,47573614332.314331,31297869554.267353
Energy Expense,38787878.787879,39169398.907104
